## Analysis

In [1]:
from pyspark.sql import SparkSession
spark_session = SparkSession.builder\
        .master("spark://192.168.2.144:7077")\
        .appName("project-demo")\
        .config("spark.cores.max", 4)\
        .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/03/11 12:29:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
reddit = spark_session.read.json("hdfs://192.168.2.144:9000/user/ubuntu/RC_2010-06_processed1.json")
print(reddit)
print(f"Number of lines {reddit.count()}")
print(reddit.take(10))

DataFrame[author: string, body: string, controversiality: bigint, is_submitter: string, score: bigint, subreddit_id: string]


Number of lines 3532867
[Row(author='indecisive11', body='Amazon is a great starting point', controversiality=0, is_submitter=None, score=0, subreddit_id='t5_2qh2a'), Row(author='illskillz', body="Oh come on. You're concluding after 4 years of the conservatives being in power that the liberals will never win a majority again? You're crazy. The leaders will change and people's values will change. Some governments will be more successful than others. ", controversiality=0, is_submitter=None, score=1, subreddit_id='t5_2qh68'), Row(author='Joe091', body="Because they didn't catch him drinking water or pissing in the shower?", controversiality=0, is_submitter=None, score=2, subreddit_id='t5_mouw'), Row(author='aoskilinn', body="My librarian friend has an MLS. Could you elaborate on what types of career paths this would open up? (Cuz I'm interested.)", controversiality=0, is_submitter=None, score=1, subreddit_id='t5_2qh1i'), Row(author='neptunesscepter', body='Topanga I wanna bang ya', contr

In [3]:
#spark_session.stop() 

to filter people who are not submitter

To filter the scores less than 2

def function(Line):
    tokenizer=Line.map(lambda Line:Line.split(' '))
    return tokenizer
lines=high_scores.function('body')

## What is the most frequent word

In [ ]:
from pyspark.sql.functions import split, lower, col
#To filter the dataframe(including columns) based on the rows with the score(2) which are not controversial
reddit_filtered = reddit.filter((col("score") == 2) & (col("controversiality") == 0)) 
#To count the frequency of each word in the column named 'body'
reddit_words = reddit_filtered.select(col("body")).rdd.flatMap(lambda x: x[0].lower().split()).map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)
#to find the most frequent word in the filtered dataframe
most_frequent_word = reddit_words.map(lambda x: (x[1], x[0])).sortByKey(False).map(lambda x: x[1]).first()

print(f'The most frequent word is "{most_frequent_word}".')

## What are the 10 most frequent words 

In [5]:
from pyspark.sql.functions import split, lower
#To filter the dataframe(including columns) based on the rows with the score(2) which are not controversial
reddit_filtered = reddit.rdd.filter(lambda row: row['score'] == 2 and row['controversiality'] == 0)
#To split the  texts inserted in the body column in to the individual words
reddit_words = reddit_filtered.flatMap(lambda row: row['body'].split())
#To count the frequency of the words
reddit_lowered = reddit_words.map(lambda word: (word.lower(), 1)).reduceByKey(lambda a, b: a + b)
#To return the 10 most frequent words
top_10 = reddit_lowered.takeOrdered(10, key=lambda x: -x[1])

for word, count in top_10:
    print(word, ' ', count)


the   829014
to   540014
a   509513
i   468030
and   423344
of   376788
you   294092
that   280040
is   279348
in   259682
